## Trying small requests

Once I have all of the data I need I can join all data frames and calculate fields needed, then I can upscale to deal with all api requests (will take a while)

In [3]:
import requests
import pandas as pd
from etl.extract.extract import extract_tickers

In [ ]:
test_ticker_list = ['AAPL', 'AACB', 'AAPD', 'ABLLL', 'HPKEW', 'HPK', 'NMRA', 'PLSE', 'PNTG', 'SEEM']
API_KEY = 'b95993f82e2d0048935cfd947df28088'


In [ ]:
# get data from financial score endpoint
score_results = []
for tick in test_ticker_list:
    url = f'https://financialmodelingprep.com/api/v4/score?symbol={tick}&apikey={API_KEY}'
    try:
        res = requests.get(url)
        data = res.json()
        if data:
            score_results.append(data[0])
    except Exception as e:
        print(f'Error fetching {tick}: {e}')
    
print(score_results)

[{'symbol': 'AAPL', 'altmanZScore': 9.097169228144661, 'piotroskiScore': 8, 'workingCapital': -25897000000, 'totalAssets': 331233000000, 'retainedEarnings': -15552000000, 'ebit': 127364000000, 'marketCap': 2987745469000, 'totalLiabilities': 264437000000, 'revenue': 400366000000}, {'symbol': 'ABLLL', 'altmanZScore': 1.2203494803590922, 'piotroskiScore': 2, 'workingCapital': 97098567, 'totalAssets': 874164752, 'retainedEarnings': -57896606, 'ebit': -2357387, 'marketCap': 796285856, 'totalLiabilities': 450870080, 'revenue': 112780530}, {'symbol': 'HPKEW', 'altmanZScore': 0.8641766808764679, 'piotroskiScore': 6, 'workingCapital': -89618000, 'totalAssets': 3063288000, 'retainedEarnings': 435834000, 'ebit': 300598000, 'marketCap': 66134748, 'totalLiabilities': 1460832000, 'revenue': 1069414000}, {'symbol': 'HPK', 'altmanZScore': 1.2476193914433868, 'piotroskiScore': 6, 'workingCapital': -89618000, 'totalAssets': 3063288000, 'retainedEarnings': 435834000, 'ebit': 300117000, 'marketCap': 10009

In [27]:

score_df = pd.DataFrame(score_results)
score_df = score_df[['symbol', 'ebit', 'workingCapital']]

In [17]:
#Get balance sheet info
balance_sheet_res = []
for tick in test_ticker_list:
    url = f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tick}?period=annual&apikey={API_KEY}'
    try:
        res = requests.get(url)
        data = res.json()
        if data:
            balance_sheet_res.append(data[0])
    except Exception as e:
        print(f'Error fetching {tick}: {e}')
        
print(balance_sheet_res)


[{'date': '2024-09-28', 'symbol': 'AAPL', 'reportedCurrency': 'USD', 'cik': '0000320193', 'fillingDate': '2024-11-01', 'acceptedDate': '2024-11-01 06:01:36', 'calendarYear': '2024', 'period': 'FY', 'cashAndCashEquivalents': 29943000000, 'shortTermInvestments': 35228000000, 'cashAndShortTermInvestments': 65171000000, 'netReceivables': 66243000000, 'inventory': 7286000000, 'otherCurrentAssets': 14287000000, 'totalCurrentAssets': 152987000000, 'propertyPlantEquipmentNet': 45680000000, 'goodwill': 0, 'intangibleAssets': 0, 'goodwillAndIntangibleAssets': 0, 'longTermInvestments': 91479000000, 'taxAssets': 19499000000, 'otherNonCurrentAssets': 55335000000, 'totalNonCurrentAssets': 211993000000, 'otherAssets': 0, 'totalAssets': 364980000000, 'accountPayables': 68960000000, 'shortTermDebt': 22511000000, 'taxPayables': 26601000000, 'deferredRevenue': 8249000000, 'otherCurrentLiabilities': 50071000000, 'totalCurrentLiabilities': 176392000000, 'longTermDebt': 96548000000, 'deferredRevenueNonCurre

In [26]:
balance_sheet_df = pd.DataFrame(balance_sheet_res)
balance_sheet_df = balance_sheet_df[['date', 'symbol', 'cashAndCashEquivalents',
                  'propertyPlantEquipmentNet', 'totalDebt']]

In [22]:
#Get company info
company_info_res = []
for tick in test_ticker_list:
    url = f'https://financialmodelingprep.com/api/v3/profile/{tick}?apikey={API_KEY}'
    try:
        res = requests.get(url)
        data = res.json()
        if data:
            company_info_res.append(data[0])
    except Exception as e:
        print(f'Error fetching {tick}: {e}')
        
print(company_info_res)

[{'symbol': 'AAPL', 'price': 209.105, 'beta': 1.275, 'volAvg': 60717452, 'mktCap': 3123150459000, 'lastDiv': 0.75, 'range': '169.21-260.1', 'changes': 10.575, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music,

In [25]:
company_info_df = pd.DataFrame(company_info_res)
company_info_df = company_info_df[['symbol', 'mktCap', 'industry', 'sector',
                 'image', 'ipoDate',
                 'isActivelyTrading']]

In [28]:
test_merge = score_df.merge(balance_sheet_df, on='symbol', how='outer')
test_merge = test_merge.merge(company_info_df, on='symbol', how='outer')
test_merge

,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,276196470,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,33465969,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3123150459000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,858860006,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True
4,HPK,3.001170e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1231674590,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True
5,HPKEW,3.005980e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,88246900,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True
6,NMRA,-2.634640e+08,2.841690e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,111638414,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1158120189,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,995464720,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True


In [29]:
# calculate fields for ranking
# Enterprise Value = market cp + total debt - cash and cash equivalents

test_merge['enterpriseValue'] = test_merge['mktCap'] + test_merge['totalDebt'] - test_merge['cashAndCashEquivalents']
test_merge



,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,276196470,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,33465969,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3123150459000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.199836e+12
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,858860006,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.112830e+09
4,HPK,3.001170e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1231674590,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.074799e+09
5,HPKEW,3.005980e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,88246900,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.313709e+08
6,NMRA,-2.634640e+08,2.841690e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,111638414,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-2.865659e+07
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1158120189,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.048980e+09
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,995464720,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.244310e+09


In [30]:
#earningsYield
#EBIT/Enterprise value

test_merge['earningsYield'] = test_merge['ebit'] / test_merge['enterpriseValue'] * 100
test_merge

,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue,earningsYield
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,276196470,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,33465969,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN,NaN
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3123150459000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.199836e+12,3.980328
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,858860006,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.112830e+09,-0.211837
4,HPK,3.001170e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1231674590,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.074799e+09,14.464874
5,HPKEW,3.005980e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,88246900,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.313709e+08,32.274790
6,NMRA,-2.634640e+08,2.841690e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,111638414,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-2.865659e+07,919.383767
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1158120189,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.048980e+09,-5.359968
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,995464720,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.244310e+09,3.012514


In [33]:
# Return On Capital

test_merge['returnOnCapital'] = (test_merge['ebit'] / (test_merge['propertyPlantEquipmentNet'] + test_merge['workingCapital'])) * 100

test_merge



,symbol,ebit,workingCapital,date,cashAndCashEquivalents,propertyPlantEquipmentNet,totalDebt,mktCap,industry,sector,image,ipoDate,isActivelyTrading,enterpriseValue,earningsYield,returnOnCapital
0,AACB,NaN,NaN,NaN,NaN,NaN,NaN,276196470,Shell Companies,Financial Services,https://images.financialmodelingprep.com/symbo...,2025-02-13,True,NaN,NaN,NaN
1,AAPD,NaN,NaN,NaN,NaN,NaN,NaN,33465969,Asset Management - Leveraged,Financial Services,https://images.financialmodelingprep.com/symbo...,2022-08-10,True,NaN,NaN,NaN
2,AAPL,1.273640e+11,-2.589700e+10,2024-09-28,2.994300e+10,4.568000e+10,1.066290e+11,3123150459000,Consumer Electronics,Technology,https://images.financialmodelingprep.com/symbo...,1980-12-12,True,3.199836e+12,3.980328,643.805287
3,ABLLL,-2.357387e+06,9.709857e+07,2024-12-31,1.319443e+08,5.747639e+06,3.859139e+08,858860006,Insurance - Life,Financial Services,https://images.financialmodelingprep.com/symbo...,2023-11-24,True,1.112830e+09,-0.211837,-2.292148
4,HPK,3.001170e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,1231674590,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-29,True,2.074799e+09,14.464874,10.876516
5,HPKEW,3.005980e+08,-8.961800e+07,2024-12-31,8.664900e+07,2.848930e+09,9.297730e+08,88246900,Oil & Gas Exploration & Production,Energy,https://images.financialmodelingprep.com/symbo...,2018-05-30,True,9.313709e+08,32.274790,10.893947
6,NMRA,-2.634640e+08,2.841690e+08,2024-12-31,1.421480e+08,2.917000e+06,1.853000e+06,111638414,Biotechnology,Healthcare,https://images.financialmodelingprep.com/symbo...,2023-09-15,True,-2.865659e+07,919.383767,-91.771804
7,PLSE,-5.622500e+07,1.093940e+08,2024-12-31,1.180380e+08,8.323000e+06,8.898000e+06,1158120189,Medical - Instruments & Supplies,Healthcare,https://images.financialmodelingprep.com/symbo...,2016-05-18,True,1.048980e+09,-5.359968,-47.762855
8,PNTG,3.748500e+07,2.115600e+07,2024-12-31,2.424600e+07,3.138820e+08,2.730910e+08,995464720,Medical - Care Facilities,Healthcare,https://images.financialmodelingprep.com/symbo...,2019-10-01,True,1.244310e+09,3.012514,11.188283
